In [1]:
import pyglet
import numpy as np
from renderer import Renderer
from enum import Enum
import random
#from colour import Color
from dmxToLED import control, dmx, Align, Modes, Color

mode = "viz"
if mode=="viz":
    from strip import Strip
else:
    from strip import StripHW as Strip



C:\Users\Brandon\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pyglet\libs\win32\__init__.py:326: UserWarning: Could not set COM MTA mode. Unexpected behavior may occur.
  warnings.warn("Could not set COM MTA mode. Unexpected behavior may occur.")


In [2]:
try: 
    gui.close()
except:
    pass

background = pyglet.image.load("cad_bg.png")

gui = Renderer(background = background, screen = pyglet.canvas.get_display().get_screens()[-1])
gui.set_location(2048, 150)

#Actual set?
stA1 = Strip(int(60*1.73))
stA2 = Strip(int(60*2.44))
stA3 = Strip(int(60*1.73))
stA = stA1 + stA2 + stA3
gui.addStrip(stA1, 547, 495, 729, 443   , r=3)
gui.addStrip(stA2, 729, 443, 1166, 443  , r=3)
gui.addStrip(stA3, 1166, 443, 1349, 495 , r=3)

stB1 = Strip(int(60*2.38))
stB2 = Strip(int(60*3.36))
stB3 = Strip(int(60*2.38))
stB = stB1 + stB2 + stB3
gui.addStrip(stB1, 400, 453, 638, 374   , r=3)
gui.addStrip(stB2, 638, 374, 1257, 374  , r=3)
gui.addStrip(stB3, 1257, 374, 1496, 453 , r=3)

stC1 = Strip(int(60*3.02))
stC2 = Strip(int(60*4.27))
stC3 = Strip(int(60*3.02))
stC = stC1 + stC2 + stC3
gui.addStrip(stC1, 256, 411, 544, 300   , r=3)
gui.addStrip(stC2, 544, 300, 1355, 300  , r=3)
gui.addStrip(stC3, 1355, 300, 1643, 411 , r=3)

stD1 = Strip(int(60*3.67))
stD2 = Strip(int(60*5.18))
stD3 = Strip(int(60*3.67))
stD = stD1 + stD2 + stD3
gui.addStrip(stD1, 110, 368, 442, 222   , r=3)
gui.addStrip(stD2, 442, 222, 1455, 222  , r=3)
gui.addStrip(stD3, 1455, 222, 1788, 368 , r=3)

fullStrip = stA+stB+stC+stD
l = float(len(fullStrip))
for i, p in enumerate(fullStrip):
    p.rgb=(1,1-(i/l),i/l)

gui.render()

patch = [
    (1, stA),
    (15, stB),
    (29, stC),
    (43, stD),
]


In [4]:
print(len(stA))
print(len(stB))
print(len(stC))
print(len(stD))

352
485
618
750


## DMX

Inputs: 
- Color 1
- Color 2
- Intens
- Mode
- Parameter
- Enable range start, stop

class Modes(Enum):
    off = 0
    solid = 1
    alternate = 2
    gradient_scroll = 3
    random_strobe = 4  #Chunks of size, at rate
    rainbow = 5
#TODO: How to center

class Align(Enum):
    left = 0
    center = 1
    right = 2


def control(strip:Strip, color1:str, color2:str, intensity:int, mode:Modes, parameter1:int, parameter2:int, startIndex:int, stopIndex:int, align:Align):
    c1 = Color(color1)
    c2 = Color(color2)

    width = parameter1*3 + 1
    offset = parameter2
    
    if align == Align.center:
        halfStripWidth = (stopIndex - startIndex)//2
        stripCenter = startIndex + halfStripWidth

    if mode == Modes.gradient_scroll:
        gradient = list(c1.range_to(c2, 128))
        gradient += gradient[::-1] #Mirror the list!
    elif mode == Modes.rainbow:
        gradient = [Color(hue=theta, saturation=1, luminance=0.5) for theta in np.linspace(0.,1., 256, endpoint=False)]

    for i, p in enumerate(strip):
        if i<int(startIndex) or i>int(stopIndex):
            p.intensity5bit=0
        else:
            p.intensity5bit = int(intensity)

            if align == Align.left:
                i = i-startIndex
            elif align == Align.center:
                i = abs(stripCenter-i)
            elif align == Align.right:
                i = stopIndex-i

            index = int((i%width)*(256.0/width))

            if mode == Modes.off:
                p.rgb8bit = (0,0,0)
            elif mode == Modes.solid:
                p.rgb = c1.rgb
            elif mode == Modes.alternate:
                if (index < (offset*2)) and ((offset*2-256)<=index):
                    p.rgb = c2.rgb
                else:
                    p.rgb = c1.rgb  
            elif mode in (Modes.gradient_scroll, Modes.rainbow):
                p.rgb = gradient[int(index-offset)].rgb
            elif mode == Modes.random_strobe:
                
                pass


patch = [
    (1, stA),
    (15, stB),
    (29, stC),
    (43, stD),
]
def dmx(universe_data, patch):
    #print(universe_data)

    
    for address, st in patch:
        data = universe_data[address-1:address+14]
        if (data[7] >= 60) or (data[0] >= 96):
            return None ##There was an error, probably bc of previs software trying to be secure
        mode = Modes(data[7] // 10)
        align = Align(data[0] // 32)

        control(st, 
                Color(f"#{data[1]:02x}{data[2]:02x}{data[3]:02x}"),
                Color(f"#{data[4]:02x}{data[5]:02x}{data[6]:02x}"),
                data[0]%32,
                mode,
                data[8],
                data[9],
                data[10]*256+data[11],
                data[12]*256+data[13],
                align
                )

## DMX things

In [3]:
import sacn
import time

patch = [
    (1, stA),
    (16, stB),
    (31, stC),
    (46, stD),
]

receiver = sacn.sACNreceiver()
receiver.start()  # start the receiving thread

@receiver.listen_on('universe', universe=1)
def callback(packet:sacn.DataPacket):
    #try:
        dmx(packet.dmxData, patch)
        #gui.render()
    #except:
    #    print("Bad data?")
    #print(packet.dmxData)  # print the received DMX data

# optional: if multicast is desired, join with the universe number as parameter
receiver.join_multicast(1)

In [4]:
while True:
    time.sleep(1/30)
    gui.render()

KeyboardInterrupt: 

In [5]:
receiver.leave_multicast(1)

receiver.stop()
gui.close()

## Development Gui

In [4]:
import PySimpleGUI as sg

st = fullStrip

stripLength = len(st)

modeRadios = [sg.Radio(m.name, "MODE", enable_events=True, key=f"-IN-mode-{m.name}",   default=False) for m in Modes]

color1 = sg.Button(button_text="Color 1", button_type=sg.BUTTON_TYPE_COLOR_CHOOSER, key="-IN-color1-")
color2 = sg.Button(button_text="Color 2", button_type=sg.BUTTON_TYPE_COLOR_CHOOSER, key="-IN-color2-")

layout = [
    [sg.Text("DMX Input Tester")],
    [sg.Text("Intensity:"),   sg.Slider(range=(0,31),           key="-IN-Intensity-", default_value=31, size=(50,15), enable_events=True, orientation="horizontal")],
    [sg.Text("Range start:"), sg.Slider(range=(0,stripLength),  key="-IN-RangeStart-",default_value=0,  size=(50,15), enable_events=True, orientation="horizontal")],
    [sg.Text("Range stop:"),  sg.Slider(range=(0,stripLength),  key="-IN-RangeStop-", default_value=65535,size=(50,15), enable_events=True, orientation="horizontal")],
    [   color1, color2],
    modeRadios,
    [sg.Checkbox("Centered mode", key="-IN-center-")],
    [sg.Text("Mode Parameter 1:"),  sg.Slider(range=(0,255), key="-IN-modeCustom1-", default_value=128,size=(50,15), enable_events=True, orientation="horizontal")],
    [sg.Text("Mode Parameter 2:"),  sg.Slider(range=(0,255), key="-IN-modeCustom2-", default_value=128,size=(50,15), enable_events=True, orientation="horizontal")],
    ]

color1.default_color = "#00ff00"
color2.default_color = "#0000ff"

window = sg.Window(title="DMX Input Parameters", layout=layout, finalize=True)

modeRadios[0].update(value=True)


while True:
    event, values = window.read(timeout = 30)
    # End program if user closes window or
    # presses the OK button
    if event == "OK" or event == sg.WIN_CLOSED:
        break

    #print(values)

    try:
        mode = Modes.off
        for m in Modes:
            if values[f"-IN-mode-{m.name}"]:
                mode = m

        control(st, 
            values["-IN-color1-"],#hexToColor(), 
            values["-IN-color2-"],#hexToColor(), 
            values["-IN-Intensity-"], 

            mode,
            values["-IN-modeCustom1-"], 
            values["-IN-modeCustom2-"], 

            values["-IN-RangeStart-"],
            values["-IN-RangeStop-"],
            center=values["-IN-center-"])
    except:
        pass
    
    gui.render()
    

window.close()

In [5]:
window.close()

In [6]:
gui.close()

In [1]:
-3%10

7

In [22]:
def c(activeWidth:int, width:int, size:int, parameter3:int):
    #Base parameters
    widthScale = 256.0/width
    shift = parameter3-128 #Its signed

    reps = activeWidth / width
    if width > reps: #If there are more pixels in each width than there are repititions, then the outer loop should be repititions
        lc = (-shift         ) %256
        uc = (-shift + size*2) %256
        lower_cutoff = int(lc/widthScale)
        upper_cutoff = int(uc/widthScale)
        invert = lc > uc%256
        if invert:
            lower_cutoff, upper_cutoff = upper_cutoff, lower_cutoff
        if invert ^ (size>=128):
            print(f"{size}  --  0:{lower_cutoff-1}\t\t{lower_cutoff}**{upper_cutoff-1}\t\t{upper_cutoff}:{width-1}\t\t{invert}")
        else:
            print(f"{size}  --  0**{lower_cutoff-1}\t\t{lower_cutoff}:{upper_cutoff-1}\t\t{upper_cutoff}**{width-1}\t\t{invert}")
    else:
        pass
        #for i in range(width):
        #    if size*2-256 <= (i*widthScale - shift)%256 < size*2:   #( - size)
        #        strip[i::width].c = rgb2
        #    else:
        #        strip[i::width].c = rgb1


for x in range(110,140, 1):
    c(300, 19, x, 0)

110  --  0:5		6**8		9:18		True
111  --  0:5		6**8		9:18		True
112  --  0:6		7**8		9:18		True
113  --  0:6		7**8		9:18		True
114  --  0:6		7**8		9:18		True
115  --  0:6		7**8		9:18		True
116  --  0:6		7**8		9:18		True
117  --  0:6		7**8		9:18		True
118  --  0:7		8**8		9:18		True
119  --  0:7		8**8		9:18		True
120  --  0:7		8**8		9:18		True
121  --  0:7		8**8		9:18		True
122  --  0:7		8**8		9:18		True
123  --  0:7		8**8		9:18		True
124  --  0:7		8**8		9:18		True
125  --  0:8		9**8		9:18		True
126  --  0:8		9**8		9:18		True
127  --  0:8		9**8		9:18		True
128  --  0:8		9**8		9:18		False
129  --  0:8		9**8		9:18		False
130  --  0:8		9**8		9:18		False
131  --  0:8		9**8		9:18		False
132  --  0:8		9**9		10:18		False
133  --  0:8		9**9		10:18		False
134  --  0:8		9**9		10:18		False
135  --  0:8		9**9		10:18		False
136  --  0:8		9**9		10:18		False
137  --  0:8		9**9		10:18		False
138  --  0:8		9**9		10:18		False
139  --  0:8		9**10		11:18		False


In [11]:
z=np.zeros(10)
z[3:2]=1
z

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])